In [ ]:
import cv2
import imutils
import numpy as np
import os
import sys
import dlib
from google.colab.patches import cv2_imshow
from PIL import Image,ImageStat

In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
main_dir = '/content/drive/MyDrive/tutorial_dml'

In [ ]:
os.chdir(main_dir)

# Alinhamento e crop das faces

In [ ]:
!wget -N http://dlib.net/files/shape_predictor_5_face_landmarks.dat.bz2

In [ ]:
!bunzip2 "shape_predictor_5_face_landmarks.dat.bz2"

In [ ]:
!ls

In [ ]:
face_file_path = main_dir+"/pessoas/IMG_20220827_155011.jpg"


In [ ]:
face_file_path.split('/')[-1]

In [ ]:
!ls

In [ ]:
if not os.path.isdir('cropped'):os.mkdir('cropped')

In [ ]:
predictor_path = "shape_predictor_5_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)

# Load the image using Dlib

def crop_images():
    for file_name in os.listdir('pessoas'):
        face_file_path = main_dir+f'/pessoas/{file_name}'
        print(face_file_path)
        img = dlib.load_rgb_image(face_file_path)

        # Ask the detector to find the bounding boxes of each face. The 1 in the
        # second argument indicates that we should upsample the image 1 time. This
        # will make everything bigger and allow us to detect more faces.
        dets = detector(img, 1)

        num_faces = len(dets)
        if num_faces == 0:
            print("Sorry, there were no faces found in '{}'".format(face_file_path))
        else:
            # Find the 5 face landmarks we need to do the alignment.
            faces = dlib.full_object_detections()
            for detection in dets:
                faces.append(sp(img, detection))



            # Get the aligned face images
            # Optionally: 
            images = dlib.get_face_chips(img, faces, size=160, padding=0.25)
            #images = dlib.get_face_chips(img, faces, size=320)
            for i,image in enumerate(images):
                #im_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
                im_pil = Image.fromarray(image)
                im_pil.save(f"cropped/{face_file_path.split('/')[-1].replace('.jpg','')}_{i}.jpg")
                

# It is also possible to get a single chip
#image = dlib.get_face_chip(img, faces[0])


In [ ]:
#crop_images()

Após o crop, separe as faces por pessoa onde cada pessoa é uma pasta

    tutorial_dml/familia/takashi
                         /patricia
                         /naomi
                         /takeo
                         /pai
                         /mae    

# Identificação das faces

## Preparação do ambiente

In [ ]:
import os



In [ ]:
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
os.chdir('/content/drive/MyDrive/tutorial_dml/')

In [ ]:
!ls

In [ ]:
!git clone https://github.com/deepinsight/insightface.git

In [ ]:
os.chdir("/content/drive/MyDrive/tutorial_dml/insightface/recognition/arcface_torch")

In [ ]:
!ls

In [ ]:
!pip install -r requirement.txt

## Carga do modelo

In [ ]:
import argparse

import cv2
import numpy as np
import torch

from backbones import get_model

In [ ]:
model = get_model('r50', fp16=False)

In [ ]:
model

In [ ]:
!gdown 17dp-EUhvX4K-g8s0ce8ODRNTiaYjTC_D

In [ ]:
model.load_state_dict(torch.load('backbone.pth'))

## Preparação do conjunto de dados

In [ ]:
import torchvision
import torchvision.transforms as transforms

In [ ]:
transform = transforms.Compose([transforms.Resize((130,130)),
                                transforms.CenterCrop((112,112)),
                                transforms.ToTensor(),
                                transforms.Normalize((0.485, 0.456, 0.406), (0.229, 0.224, 0.225))
                            ])

In [ ]:
ds = torchvision.datasets.ImageFolder("/content/drive/MyDrive/tutorial_dml/familia3",transform=transform)

In [ ]:
len(ds)

In [ ]:
dl = torch.utils.data.DataLoader(ds,batch_size=len(ds))

In [ ]:
model.eval()

In [ ]:
x,y = next(iter(dl))

In [ ]:
x.shape

## Construção dos embeddings

In [ ]:
pred = model(x)

In [ ]:
pred.shape

In [ ]:
xall = np.asarray(pred.tolist())

## Splits de treino e teste

In [ ]:
from sklearn.model_selection import train_test_split

In [ ]:
xtrain, xtest, ytrain, ytest = train_test_split(xall, y, test_size=0.33, random_state=42,stratify=np.array(y))

## Treino e execução do knn

In [ ]:
import sklearn.neighbors

In [ ]:
knn = sklearn.neighbors.KNeighborsClassifier(n_neighbors=1,weights='distance')

In [ ]:
knn.fit(xtrain,ytrain)

In [ ]:
pred = knn.predict(xtest)

## Avaliação do modelo

In [ ]:
import sklearn.metrics as metrics

In [ ]:
print(metrics.classification_report(ytest,pred))

In [ ]:
ds.classes

In [ ]:
print(metrics.confusion_matrix(ytest,pred))

# Chatbot

In [ ]:
!pip install python-telegram-bot --upgrade

In [ ]:
from telegram.ext import Updater, Filters, MessageHandler, CommandHandler
import requests
import re
import torchvision.transforms as transforms
from PIL import Image,ImageStat


In [ ]:
knn.fit(xall, y)

In [ ]:
predictor_path = main_dir+ "/shape_predictor_5_face_landmarks.dat"
detector = dlib.get_frontal_face_detector()
sp = dlib.shape_predictor(predictor_path)

# Load the image using Dlib

def crop_images(image_file):

    face_file_path = image_file
    print(face_file_path)
    img = dlib.load_rgb_image(face_file_path)

    # Ask the detector to find the bounding boxes of each face. The 1 in the
    # second argument indicates that we should upsample the image 1 time. This
    # will make everything bigger and allow us to detect more faces.
    dets = detector(img, 1)
    crop_found = []
    num_faces = len(dets)
    if num_faces == 0:
        print("Sorry, there were no faces found in '{}'".format(face_file_path))
    else:
        # Find the 5 face landmarks we need to do the alignment.
        faces = dlib.full_object_detections()
        for detection in dets:
            faces.append(sp(img, detection))



        # Get the aligned face images
        # Optionally: 
        images = dlib.get_face_chips(img, faces, size=160, padding=0.25)
        #images = dlib.get_face_chips(img, faces, size=320)
        for i,image in enumerate(images):
            #im_pil = Image.fromarray(cv2.cvtColor(image, cv2.COLOR_BGR2RGB))
            im_pil = Image.fromarray(image)
            crop_found.append(im_pil)
    return crop_found

In [ ]:
file_name = "IMG_20220827_155011.jpg"

In [ ]:
crops = crop_images("/content/drive/MyDrive/tutorial_dml/pessoas/"+file_name)

In [ ]:
nomes = np.asarray(ds.classes)

In [ ]:
def get_person_ids(crops):
    imglist = []
    for img in crops:
        imglist.append(transform(img).unsqueeze(0))
    x = torch.cat(imglist,axis=0)
    embeddings = model(x)
    embeddings = np.asarray(embeddings.tolist())
    prob_pessoas = knn.predict_proba(embeddings)
    id_pessoas = prob_pessoas.argmax(axis=1)
    return nomes[id_pessoas]


In [ ]:
pessoas_identificadas=get_person_ids(crops)

In [ ]:
print(f'{pessoas_identificadas}')

In [ ]:
len(crops)

In [ ]:
nomes = np.asarray(ds.classes)

In [ ]:
dir_bot = "/content/drive/MyDrive/tutorial_dml/imgs_recebidas"

In [ ]:
dir_pessoas = "/content/drive/MyDrive/tutorial_dml/imgs_pessoas"

In [ ]:
if not os.path.isdir(dir_bot): os.mkdir(dir_bot)
if not os.path.isdir(dir_pessoas): os.mkdir(dir_pessoas)

In [ ]:
def encontra_pessoas(image_file):
    crops = crop_images(dir_bot+os.sep+image_file)
    for i,img in enumerate(crops):
        img.save(dir_pessoas+f"/{i}.jpg")

    return get_person_ids(crops)

In [ ]:
import pdb

In [ ]:



def image_handler(update, context):
    file = update.message.photo[-1].file_id
    obj = context.bot.get_file(file)
    os.chdir(dir_bot)
    path = obj.download()
    pessoas_identificadas = encontra_pessoas(path)
    print(pessoas_identificadas)
    for i, nome_pessoa in enumerate(pessoas_identificadas):
        update.message.reply_photo(open(dir_pessoas+f"/{i}.jpg","rb"))
        update.message.reply_text(f'{nome_pessoa}')

def start(update, context):
  return update.message.reply_text('Seja bem vindo ao reconhecedor facial ')

def main():
    keytelegram = "XXXX" # criar a sua key
    updater = Updater(keytelegram)
    dp = updater.dispatcher
    dp.add_handler(MessageHandler(Filters.photo, image_handler))
    dp.add_handler(CommandHandler('start', start))
    updater.start_polling()
    updater.idle()

In [ ]:
main()